# Facebook_Bidding_AB_Testing

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import shapiro, levene, ttest_ind
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 10)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [3]:
#Importing Data
Control_Group = pd.read_excel("/Users/yurtcanmustafa/Desktop/DataScience/ab_testing.xlsx", sheet_name = 'Control Group')  # maximum bidding
Test_Group = pd.read_excel("/Users/yurtcanmustafa/Desktop/DataScience/ab_testing.xlsx", sheet_name = 'Test Group')        # average bidding

In [4]:
def check_df(dataframe, head=5):
    print("### Head ###")
    print(dataframe.head(head))
    print("### Tail ###")
    print(dataframe.tail(head))
    print("###Shape ###")
    print(dataframe.shape)
    print("### Types ###")
    print(dataframe.dtypes)
    print("### NA ###")
    print(dataframe.isnull().sum())
    print("### Describe ###")
    print(dataframe.describe().T)

In [5]:
check_df(Control_Group)

### Head ###
   Impression    Click  Purchase  Earning
0   82529.459 6090.077   665.211 2311.277
1   98050.452 3382.862   315.085 1742.807
2   82696.024 4167.966   458.084 1797.827
3  109914.400 4910.882   487.091 1696.229
4  108457.763 5987.656   441.034 1543.720
### Tail ###
    Impression    Click  Purchase  Earning
35  132064.219 3747.158   551.072 2256.976
36   86409.942 4608.256   345.046 1781.358
37  123678.934 3649.074   476.168 2187.721
38  101997.494 4736.353   474.614 2254.564
39  121085.881 4285.179   590.406 1289.309
###Shape ###
(40, 4)
### Types ###
Impression    float64
Click         float64
Purchase      float64
Earning       float64
dtype: object
### NA ###
Impression    0
Click         0
Purchase      0
Earning       0
dtype: int64
### Describe ###
            count       mean       std       min       25%       50%  \
Impression 40.000 101711.449 20302.158 45475.943 85726.690 99790.701   
Click      40.000   5100.657  1329.985  2189.753  4124.304  5001.221   
Purcha

In [6]:
check_df(Test_Group)

### Head ###
   Impression    Click  Purchase  Earning
0  120103.504 3216.548   702.160 1939.611
1  134775.943 3635.082   834.054 2929.406
2  107806.621 3057.144   422.934 2526.245
3  116445.276 4650.474   429.034 2281.429
4  145082.517 5201.388   749.860 2781.698
### Tail ###
    Impression    Click  Purchase  Earning
35   79234.912 6002.214   382.047 2277.864
36  130702.239 3626.320   449.825 2530.841
37  116481.873 4702.782   472.454 2597.918
38   79033.835 4495.428   425.359 2595.858
39  102257.454 4800.068   521.311 2967.518
###Shape ###
(40, 4)
### Types ###
Impression    float64
Click         float64
Purchase      float64
Earning       float64
dtype: object
### NA ###
Impression    0
Click         0
Purchase      0
Earning       0
dtype: int64
### Describe ###
            count       mean       std       min        25%        50%  \
Impression 40.000 120512.412 18807.449 79033.835 112691.971 119291.301   
Click      40.000   3967.550   923.095  1836.630   3376.819   3931.360   


In [8]:
Control_Group.columns = [col+"_Control" for col in Control_Group.columns]
Control_Group.head()

,Impression_Control,Click_Control,Purchase_Control,Earning_Control
0,82529.459,6090.077,665.211,2311.277
1,98050.452,3382.862,315.085,1742.807
2,82696.024,4167.966,458.084,1797.827
3,109914.400,4910.882,487.091,1696.229
4,108457.763,5987.656,441.034,1543.720


In [9]:
Test_Group.columns = [col+"_Test" for col in Test_Group.columns]
Test_Group.head()

,Impression_Test,Click_Test,Purchase_Test,Earning_Test
0,120103.504,3216.548,702.160,1939.611
1,134775.943,3635.082,834.054,2929.406
2,107806.621,3057.144,422.934,2526.245
3,116445.276,4650.474,429.034,2281.429
4,145082.517,5201.388,749.860,2781.698


In [10]:
df = pd.concat([Control_Group, Test_Group], axis=1)
df.head()

,Impression_Control,Click_Control,Purchase_Control,Earning_Control,Impression_Test,Click_Test,Purchase_Test,Earning_Test
0,82529.459,6090.077,665.211,2311.277,120103.504,3216.548,702.160,1939.611
1,98050.452,3382.862,315.085,1742.807,134775.943,3635.082,834.054,2929.406
2,82696.024,4167.966,458.084,1797.827,107806.621,3057.144,422.934,2526.245
3,109914.400,4910.882,487.091,1696.229,116445.276,4650.474,429.034,2281.429
4,108457.763,5987.656,441.034,1543.720,145082.517,5201.388,749.860,2781.698


# AB Testing (Independent Two-Sample T-Test)

Indepented Two Sample T-Test

The Independent Samples t Test compares the means of two independent groups in order to determine whether there is statistical evidence that the associated population means are significantly different.


Requirements

1. Assumption Check

* Normality Assumption: Non-normal population distributions, especially those that are thick-tailed or heavily skewed, considerably reduce the power of the test

* Variance Homogeneity: When this assumption is violated and the sample sizes for each group differ, the p value is not trustworthy.

2. Implementation of the Hypothesis

* Independent two-sample t-test if assumptions are met (parametric test)
* Mannwhitneyu test if assumptions are not provided (non-parametric test) 

Note:
* If the normality is not provided, the mannwhitneyu test should be applied directly.

* If the normality assumption is provided but the variance homogeneity is not provided, the equal_var parameter can be set to False for the two-sample t-test.

In [13]:
A = pd.DataFrame(Control_Group["Purchase_Control"])
B = pd.DataFrame(Test_Group["Purchase_Test"])

A.mean() # Maximum Bidding

Purchase_Control   550.894
dtype: float64

In [14]:
B.mean() # Average Bidding

Purchase_Test   582.106
dtype: float64

# Hypothesis

The null hypothesis (H0) and alternative hypothesis (H1) of the Independent Samples t Test can be expressed in two different but equivalent ways:

H0: µ1 = µ2 (the two population means are equal)

H1: µ1 ≠ µ2 (the two population means are not equal)


For this study:

H0 : There is no statistically significant difference between the Control group that was served “maximum bidding” campaign and Test group that was served “average bidding” campaign. (µ1 = µ2)

H1 : There is statistically significant difference between the Control group that was served “maximum bidding” campaign and Test group that was served “average bidding” campaign. (µ1 != µ2)

# Assumption Check

1. Normality Assumption
   
The Shapiro-Wilks Test for Normality

    * H0: There is no statistically significant difference between sample distribution and theoretical normal distribution

    * H1: There is statistically significant difference between sample distribution and theoretical normal distribution

The test rejects the hypothesis of normality when the p-value is less than or equal to 0.05. 
Failing the normality test allows you to state with 95% confidence the data does not fit the normal distribution.

p-value < 0.05 (H0 rejected)

p-value > 0.05 (H0 not rejected)


For this study:

    * H0: There is no difference between the number of products purchased. (p-value < 0.05)
    
    * H1: Normal distribution assumption not provided.

In [17]:
test_stat, pvalue = shapiro(A["Purchase_Control"])
print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))

Test Stat = 0.9773, p-value = 0.5891


P-value is greater than 0.05, we can't reject Ho hypothesis. 

For this reason, we can say it is the normal distribution for the 1st group.

In [18]:
test_stat, pvalue = shapiro(B["Purchase_Test"])
print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue)) 

Test Stat = 0.9589, p-value = 0.1541


P-value is greater than 0.05, we can't reject Ho hypothesis. 

For this reason, we can say it is the normal distribution for the 2nd group.

Lastly, there is no statistically significant difference between sample distribution and theoretical normal distribution in groups A and B.


2. Variance Homogeneity Assumption

Levene’s Test for Homogeneity of Variances

Levene’s test is an equal variance test. 
It can be used to check if our data sets fulfill the homogeneity of variance assumption before we perform the t-test or Analysis of Variance

    * H0: the compared groups have equal variance.
    
    * H1: the compared groups do not have equal variance.


For this study:

    * H0: Variances are Homogeneous

    * H1: Variances are not Homogeneous

In [19]:
test_stat, pvalue = levene(A["Purchase_Control"],
                           B["Purchase_Test"].dropna())

print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))

Test Stat = 2.6393, p-value = 0.1083


P-value is greater than 0.05, we can't reject Ho hypothesis.

The compared groups have equal variance.

The assumptions of normality distribution and variance homogeneity were tested.

Two assumptions are provided, now we can test for our main hypothesis.

In our study, since our assumumption are made, we should apply parametric test(the independent two-sample t test). 


In [20]:
test_stat, pvalue = ttest_ind(A["Purchase_Control"],
                              B["Purchase_Test"],
                              equal_var=True)

print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))

Test Stat = -0.9416, p-value = 0.3493


P-value greater then 0.05 so H0 is not rejected.

So, there is no statistically significant difference between the Control group that was served “maximum bidding” campaign and Test group that was served “average bidding” campaign.

For this reason, I can recommend to Facebook continuing with the maximum bidding campaign currently used.